In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)
    print("Polarity:\n", text_polarity)
    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment

def get_losers():
    
    print('Losers'.center(45))
    print()
    print('Symbol'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Loss')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers("losers"))):
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc)+'%')
def get_gainers():
    print('Gainers'.center(45))
    print()
    print('Symbol'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Gain')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers())):
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc)+'%')

if __name__ == '__main__':
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    print("{:<30}".format('–' * 45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    print("{:<30}".format('–' * 45))
    bears = selector.bearish_candlesticks(64, 20)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ben/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                   Gainers                   

Symbol     Last        Change      Pct Gain
–––––––––––––––––––––––––––––––––––––––––––––
SMTX       $3.13       $0.78       33.191%
CUI        $0.92       $0.21       29.577%
TRXC       $0.28       $0.053      23.566%
PHIO       $0.3        $0.05       19.952%
CARB       $21.6999    $3.3        17.934%
NXTD       $0.5001     $0.061      13.918%
DGAZ       $107.15     $11.51      12.035%
TRNX       $1.94       $0.18       10.227%
HTBX       $0.54       $0.05       10.204%
CNTG       $14.25      $1.26       9.7%
CNSP       $4.96       $0.39       8.534%
DARE       $0.9        $0.069      8.316%
JMU        $1.2        $0.09       8.108%
MPW        $21.54      $1.59       7.97%
SLS        $6.2        $0.45       7.826%
NKTR       $21.52      $1.48       7.385%
KOLD       $26.71      $1.81       7.269%
FWONA      $40.65      $2.71       7.143%
PVG        $9.49       $0.62       6.99%
PIR        $7.2        $0.47       6.984%
ADMP       $1.08  

In [25]:
ticker = "TRXC"

In [26]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
# stocktwitstext

In [27]:
tweets = ti.get_tweets(ticker, "stock")
tweets

'https://t.co/0ulZdbuMyc Three Simple Stock Patterns for Success!!! $TLRY $TRXC $NBEV\n\nhttps://t.co/2IgJ7sW3kY  How to eliminate cognitive biases from stock trading! $FB $TWTR $TRXC $LBIX\n\nhttps://t.co/2IgJ7sW3kY  How to eliminate cognitive biases from stock trading! $FB $TWTR $TRXC $LBIX\n\n$TRXC short ratio on 2019-05-31 is 11.33 https://t.co/dUIWBvHl7q $SYK 2.59 $JNJ 1.87 $CMD 9.09\n\nEvening Most Tweeted Micro Caps, check out Gambiste Top 10: $SOL $TRXC $WATT $PIRS $TEUM $PYX $ROSE $AGRX $WKHS https://t.co/k3uyuEbRpJ\n\nhttps://t.co/TVlmFwpcHE &lt; Jason Bond is a trading GENIUS!! $BKNG $TRXC $QCOM\n\nhttps://t.co/0ulZdbuMyc Three Simple Stock Patterns for Success!!! $TLRY $TRXC $NBEV\n\n$TRXC closes flat at 0.0%: https://t.co/5bvEjIbk52\nSentiment: Neutral #TRXC #TransEnterix\n\nTransEnterix $NYSEAMERICAN:TRXC Stock Price Down 10.7%  https://t.co/nngqy8jw1X\n\nTransEnterix $NYSEAMERICAN:TRXC Stock Price Down 10.7% https://t.co/M5yYQ0Gl0p\n\nTransEnterix $NYSEAMERICAN:TRXC Stoc

In [28]:
stsentiment = get_sentiment(stocktwitstext)
tsentiment = get_sentiment(tweets)
print("\nSentiments\n\nStocktwits: {}\nTwitter: {}".format(stsentiment, tsentiment))

Polarity:
 {'neg': 0.098, 'neu': 0.769, 'pos': 0.133, 'compound': 0.8749}
Polarity:
 {'neg': 0.014, 'neu': 0.906, 'pos': 0.081, 'compound': 0.913}

Sentiments

Stocktwits: positive
Twitter: positive


In [29]:
alpaca_api.polygon.historic_trades(ticker, '2019-10-10', limit=10)
alpaca_api.polygon.news(ticker)
alpaca_api.polygon.last_trade(ticker)
alpaca_api.polygon.last_quote(ticker)
alpaca_api.polygon.company(ticker)
alpaca_api.polygon.dividends(ticker)
alpaca_api.polygon.financials(ticker)

[Financial({   'cashChange': -56046000,
     'cashFlow': -16451000,
     'costOfRevenue': 4249000,
     'currentAssets': 106419000,
     'currentCash': 42411000,
     'currentDebt': None,
     'grossProfit': 1173000,
     'netIncome': -20206000,
     'operatingExpense': 12665000,
     'operatingGainsLosses': 8804000,
     'operatingIncome': -11492000,
     'operatingRevenue': 5422000,
     'reportDate': '2018-09-30T00:00:00.000Z',
     'reportDateStr': '2018-09-30',
     'researchAndDevelopment': 4838000,
     'shareholderEquity': 164063000,
     'symbol': 'TRXC',
     'totalAssets': 227559000,
     'totalCash': 81418000,
     'totalDebt': None,
     'totalLiabilities': None,
     'totalRevenue': 5422000}), Financial({   'cashChange': 4246000,
     'cashFlow': -6891000,
     'costOfRevenue': 3732000,
     'currentAssets': 118340000,
     'currentCash': 98493000,
     'currentDebt': None,
     'grossProfit': 2657000,
     'netIncome': -34248000,
     'operatingExpense': 18509000,
     '

In [30]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)
tenquant_data

{'error': 'could not find data.'}